In [ ]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot

hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook()


## Draw gnomad controls

In [ ]:
gnomad = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gnomad/gnomad-ready-to-draw.ht')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-pheno-cadd.mt')

In [ ]:
gd = hl.MatrixTable.from_rows_table(gnomad)

In [ ]:
gd = gd.annotate_cols(col_idx = hl.null(hl.tint32))

In [ ]:
gd = gd.key_cols_by(gd.col_idx)

In [ ]:
gnomad_samples = []

for i in range(1,186):
    gnomad_samples.append(str(i)+'_gnomad')

In [ ]:
gnomad_columns = hl.utils.range_matrix_table(n_rows=0, n_cols=185)

In [ ]:
gnomad_columns = gnomad_columns.annotate_rows(locus = hl.locus(hl.null(hl.tstr), hl.null(hl.tint32), reference_genome='GRCh38'),
                                              alleles = hl.null(hl.tarray(hl.tstr)))

In [ ]:
gnomad_columns = gnomad_columns.key_rows_by(gnomad_columns.locus, gnomad_columns.alleles)

In [ ]:
gd = gd.union_cols(gnomad_columns, row_join_type='outer')

In [ ]:
gd = gd.annotate_entries(AD = hl.null(hl.tarray(hl.tint32)),
                         DP = hl.null(hl.tint32),
                         GQ = hl.null(hl.tint32),
                         GT = hl.call(hl.int32(hl.rand_bool(gd.v3_nfe['AF'])), hl.int32(hl.rand_bool(gd.v3_nfe['AF']))),
                         MIN_DP = hl.null(hl.tint32),
                         PGT = hl.call(hl.int32(hl.rand_bool(gd.v3_nfe['AF'])), hl.int32(hl.rand_bool(gd.v3_nfe['AF']))),
                         PID = hl.null(hl.tstr),
                         PL = hl.null(hl.tarray(hl.tint32)),
                         PS = hl.null(hl.tint32),
                         RGQ = hl.null(hl.tint32),
                         SB = hl.null(hl.tarray(hl.tint32)))

In [ ]:
gd.write('/net/archive/groups/plggneuromol/GTS-analysis/gnomad/test.ht') #this directory should not exist